In [1]:
import argparse

In [2]:
%run model.ipynb

/Users/britnybrito/opt/anaconda3/lib/python3.9/site-packages/torch/amp/autocast_mode.py:204: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


aaaaaa QT
196
torch.Size([10, 300, 196])
torch.Size([10, 301, 196])
torch.Size([1, 301, 196])
torch.Size([10, 50])
aaaaaa QT
196
torch.Size([10, 108, 196])
torch.Size([10, 109, 196])
torch.Size([1, 109, 196])
torch.Size([10, 527])


In [3]:
%run dataloader.ipynb
import csv

In [4]:
labels = 'class_labels_indices.csv'
audio_conf = {'num_mel_bins': 128, 'target_length': 1024, 'freqm': 24, 'timem': 192, 'mixup': 0.5}
ayuda = AudiosetDataset('test_data.json', audio_conf, label_csv = labels)

number of classes is 75


In [5]:
train_loader = torch.utils.data.DataLoader(ayuda)





In [6]:
ast_mdl = ASTModel( class_n=75)

aaaaaa QT
196


In [7]:
import sys
import os
import datetime
sys.path.append(os.path.dirname(os.path.dirname(sys.path[0])))
import time
import torch
from torch import nn
import numpy as np
import pickle
from torch.cuda.amp import autocast,GradScaler

In [8]:
for i in ayuda:
    print(i)

KeyError: 'T002'

In [ ]:
for i, (audio_input, labels) in enumerate(train_loader):
    audio_input = audio_input.to(device, non_blocking=True)
    with autocast():
        audio_output = ast_mdl(audio_input)
                

In [ ]:
ast_mdl(ayuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")